In [1]:
# 这个格子是生成短链接的部分
import requests

def get_short_url(url, data=None):
    baseurl = 'http://api.t.sina.com.cn/short_url/shorten.json?source=3271760578&url_long='
    url = baseurl + url
    rep = requests.get(url, timeout=60)
    return rep.json()[0]['url_short']

In [2]:
# 这个格子是数据库的配置

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker


DB_HOST = 'rds1900ea64zb9dk510v.mysql.rds.aliyuncs.com'
DB_USER = 'medserver'
DB_PWD = 'Medapp2017!@#'
DB_NAME = 'medapp'

Base = declarative_base()
engine = create_engine('mysql+mysqlconnector://%s:%s@%s/%s?charset=utf8' % (DB_USER, DB_PWD, DB_HOST, DB_NAME), encoding='utf-8',
                       echo=False, pool_size=100, pool_recycle=10)
db = scoped_session(sessionmaker(bind=engine, autocommit=False, autoflush=True, expire_on_commit=False))

In [6]:
from collections import namedtuple

SMS_Content = namedtuple('SMS_Content', ['name', 'phone', 'content'])


In [13]:
print(get_short_url('http://cdnoss.medcircle.cn/app/qrcode/0_52390_1556265361536_82928.png'))

http://suo.im/4nPSgb


In [22]:
# 下面是获取数据通过mysql来获取数据

from time import sleep
from openpyxl import Workbook

wb = Workbook()#建立一个工作本
ws = wb.active#激活sheet
ws.title='短信数据'#对sheet进行命名
ws.append(['姓名', '手机号', '内容'])
sql_str = """
select reg_barcode, reg_phone, reg_name from con_register where reg_cid=3142;
"""
results = db.execute(sql_str).fetchall()
if results:
    for result in results:
        if result[1]:
            content = get_short_url(result[0])
            ws.append([result[2], result[1], content])
wb.save("短信内容.xlsx")

# 下面是群发短信的功能，发送之前一定要查看账户里面短信的条数是否够用

In [2]:
# 这个地方是配置的内容
xlsxPath = '/Users/apple/Desktop/短信内容.xlsx'       # 这里配置xlsx的文件路径
sms_sign = '【会务组】'        # 短信的签名
phone_lab= '手机号'            # 对应的短信的列名
name_lab = '姓名'              # 对应的姓名的列名
content_lab = '内容'           # 对应的内容列的列名
split_num = 500        # 一次发短信的条数
new_content = """你好%s:你的注册内容是"""      # 要发送的内容
content_replace_str=('姓名',)                  # 要替换的内容，如果这个地方没有，那么上面的new_content不走，只会走content_lab列的内容
sms_sn = "SDK-BBX-010-28180"                  # 短信的账号
sms_pwd = "1302551C14838E916EFEE9A359D93204"        # 这个地方是后台通过java代码调用漫道的md5(sn + pwd)而成

In [ ]:
# 读取数据
from openpyxl import load_workbook
from collections import namedtuple
workbook = load_workbook(xlsxPath)
#booksheet = workbook.active                #获取当前活跃的sheet,默认是第一个sheet
sheets = workbook.get_sheet_names()         #从名称获取sheet
booksheet = workbook.get_sheet_by_name(sheets[0])
has_replace = bool(content_replace_str)
rows = booksheet.rows
titleList = None
contentList = []
for row in rows:
    if not titleList:
        # 通过第一行的标题生成Record对象
        line = [col.value for col in row if col.value]
        titleList = line
        # 通过namedtuple声称对象
        # print(titleList)
        Record = namedtuple('Record', titleList)
    else:
        # 先进行检查是否到了终止的地方
        if row[0].value:
            # 有数据
            singleList = [col.value for col in row[:len(titleList)]]
            if has_replace:
                record = Record._make(singleList)
                singleList[titleList.index(content_lab)] = new_content % tuple((getattr(record, str(s)) for s in content_replace_str))
            record = Record._make(singleList)
            # print(record)
            contentList.append(record)
        else:
            break;

In [3]:
# 下面是发短信的方法
import requests
import re

def send_sms(phone, content):
    """
    发短信
    :param phone: 手机号
    :param content: 内容
    :return: 如果成功，返回匹配的内容
    """
    sms_url = "http://sdk.entinfo.cn:8060/webservice.asmx"
    soap_action = "http://tempuri.org/gxmt"
    xml = """<?xml version="1.0" encoding="utf-8"?>
    <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 
    xmlns:xsd="http://www.w3.org/2001/XMLSchema" 
    xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
    <soap:Body>
    <gxmt xmlns="http://tempuri.org/">
    <sn>%s</sn>
    <pwd>%s</pwd>
    <mobile>%s</mobile>
    <content>%s</content>
    <ext>%s</ext>
    <stime>%s</stime>
    <rrid>%s</rrid>
    </gxmt>
    </soap:Body>
    </soap:Envelope>"""
    if phone and content:
        send_xml = xml % (sms_sn, sms_pwd, phone, content, "",
                          "", "")
        head = {'Content-Type': 'text/xml; charset=utf-8', 'SOAPAction': soap_action}
        response_text = requests.post(sms_url, headers=head, data=send_xml.encode('utf-8'))
        print(response_text.text)
        try:
            search_result = re.search(r"<gxmtResult>(.*)</gxmtResult>", str(response_text.text), 0)
            if search_result:
                print(phone + "   发送成功")
            else:
                print(phone + "   发送失败")
        except Exception as e:
            print(phone + "   发送失败")

In [4]:
send_sms('15611332279', '测试内容' + sms_sign)

<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><gxmtResponse xmlns="http://tempuri.org/"><gxmtResult>242313590093705683</gxmtResult></gxmtResponse></soap:Body></soap:Envelope>
15611332279   发送成功
15611332279   发送失败


In [41]:
result_time = int(len(results) / split_num) + 1
print("results的个数：" + str(len(results)))
print("发送的次数：" + str(result_time))

results的个数：1275
发送的次数：3


In [46]:
test_str = ''
print(test_str[0: -1])

16735167245,18718346741


In [ ]:
if result_time == 1:
    phone = ''
    content = ''
    for record in contentList:
        phone += (str(getattr(record, phone_lab)) + ',')
        content += (str(getattr(record, content_lab))  + sms_sign + ',')
    if phone: 
        send_sms(phone[0: -1], content[0: -1])
else:
    for i in range(result_time):
        phone = ''
        content = ''
        for record in contentList[i * split_num : (i + 1) * split_num]:
            phone += (str(getattr(record, phone_lab)) + ',')
            content += (str(getattr(record, content_lab)) + sms_sign + ',')
        if phone:
            send_sms(phone[0: -1], content[0: -1])

In [1]:
import re
test_str = """
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><gxmtResponse xmlns="http://tempuri.org/"><gxmtResult>242257401705949981</gxmtResult></gxmtResponse></soap:Body></soap:Envelope>
"""
search_result = re.search(r"<gxmtResult>(.*)</gxmtResult>", str(test_str), 0)
if search_result:
    print("找到了")
else:
    print("没找到")

找到了


In [2]:
# 读取数据
from openpyxl import load_workbook, Workbook


sql_str = """
select reg_barcode from con_register where reg_name='%s' and reg_phone='%s';
"""
workbook = load_workbook('筛选.xlsx')
#booksheet = workbook.active                #获取当前活跃的sheet,默认是第一个sheet
sheets = workbook.sheetnames         #从名称获取sheet
#print(sheets)
booksheet = workbook[sheets[0]]
contentList = []
rows = booksheet.rows
for row in rows:
    
    # 先进行检查是否到了终止的地方
    if row[6].value and row[6].value != '照片':
        contentList.append(row[6].value)
        # 有数据
        #print(row[12].value)
        #results = db.execute(sql_str % (row[1].value,row[3].value)).fetchall()
        #print(results)
        #print(results)
        #if results:
        #    contentList.append(results[0][0])
            #ws.append(get_short_url(results[0][0]))
        #else:
            #ws.append("")
            #print("无")
    #else:
    #    break;
#wb.save('短信内容.xlsx')

In [16]:
sql_str = """
select reg_barcode from con_register where reg_code=%s;
"""
results = db.execute(sql_str % '4103142058161').fetchall()
print(type(results[0][0]))

<class 'str'>


In [5]:
wb = Workbook()#建立一个工作本
ws = wb.active#激活sheet
ws.title='短信数据'#对sheet进行命名
ws.append(['内容'])
for str_url in contentList:
    ws.append([get_short_url(str_url)])
wb.save('短信内容.xlsx')

In [3]:
contentList

['http://cdnoss.medcircle.cn/app/icon/8_0_0_1558790278318_85391.jpeg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558787299865_22172.jpg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558758210115_15260.jpg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558751162585_16561.jpeg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558749351918_49403.jpeg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1557801382414_26826.jpeg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558746279983_74174.jpeg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558713660198_81815.jpg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558707358235_22986.jpg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558699837552_75589.JPG',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558691461941_11880.jpeg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558690037932_60794.jpeg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558685094197_49319.jpeg',
 'http://cdnoss.medcircle.cn/app/icon/8_0_0_1558684309494_48435.jpg',
 'http://cdn